## Text extraction from OCR scanned Handwritten Slave Trade Volumes
Supervisor: Prof. Henry Lovejoy for the Digital Slavery Research Lab

Author: Sushma Akoju (Sushma Anand Akoju)

### Class A Document Analysis and Text Extractions from Slave Trade Volumnes
### British Parliamentary Papers

Install Gcloud, google python api for OAuth i.e Authentication tokens, Google Cloud and Google Cloud Document AI packages. 

- Remember you will have to Restart Colab session after this installation step.

In [ ]:
!pip install gcloud
!pip install google-api-python-client
!pip install google-cloud
!pip install google-cloud-documentai

In [ ]:
# NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
# NotebookApp.rate_limit_window=3.0 (secs)

Create test Authentication method.

In [ ]:
from google.cloud import documentai_v1beta3
import pprint

def sample_create_processor():
    # Create a client
    client = documentai_v1beta3.DocumentProcessorServiceClient()

    # Initialize request argument(s)
    request = documentai_v1beta3.CreateProcessorRequest(
        parent="parent_value",
    )

    # Make the request
    response = client.create_processor(request=request)

    # Handle the response
    pprint.pprint(response)

In [ ]:
# sample_create_processor()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/slave-trade-volumes/all/
!ls

/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all
'Slave Trade Volume 10'  'Slave Trade Volume 11'  'Slave Trade Volume 12'


In [ ]:
import os
path = '/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/'
volumes_dict = {folder : os.path.join(path,folder) for folder in os.listdir(path) if "Slave Trade Volume 12" not in folder}

volumes = [os.path.join(path,folder) for folder in os.listdir(path) if "Slave Trade Volume 12" not in folder]

classa_docs = {volumes[1]: ['Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1824-1825',
               'Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1825-1826'],
               volumes[0]:['Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1826-1827',
               'Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1827']}
classb_docs = {volumes[1]: ['Class B. Correspondence with foreign powers, relating to the slave trade. 1824-1825',
               'Class B. Correspondence with foreign powers, relating to the slave trade. 1825-1826'],
               volumes[0]: ['Class B. Correspondence with foreign powers, relating to the slave trade. 1826-1827',
               'Class B. Correspondence with foreign powers, relating to the slave trade. 1827']}

classa_collections = { os.path.basename(volume) : {
    doc: [os.path.join(volume,"tenpages/classa",coll) for coll in os.listdir(os.path.join(volume, "tenpages/classa")) if doc in coll]
    for doc in classa_docs[volume]} for volume in volumes}

classb_collections = { os.path.basename(volume) : {
    doc : [os.path.join(volume,"tenpages/classb",coll) for coll in os.listdir(os.path.join(volume, "tenpages/classb")) if doc in coll]
    for doc in classb_docs} for volume in volumes}
tocs = { os.path.basename(volume) : [os.path.join(volume,"tenpages/",coll) for coll in os.listdir(os.path.join(volume, "tenpages")) if "toc" in coll] for volume in volumes}

#classa_collections, classb_collections, tocs

In [ ]:
partition_counters =  {}
for vol, docs in classa_collections.items():
  for doc_name, file_paths in docs.items():
    partition_counters[doc_name] = 0
    for file_path in file_paths:
      if doc_name in file_path:
        partition_counters[doc_name] += 1

In [ ]:
partition_counters

{'Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1824-1825': 19,
 'Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1825-1826': 10,
 'Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1826-1827': 18,
 'Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1827': 17}

Ensure you configured your account with your API: [Setup Document AI](https://cloud.google.com/document-ai/docs/setup )

Here we process each of 10-page partitions of each of Collection from each Slave Trade Volume.

In [ ]:

# TODO(developer): Uncomment these variables before running the sample.
project_id= 'theorem-prover-4182022'
location = 'us' # Format is 'us' or 'eu'
processor_id = '38d99862def047aa' # Create processor in Cloud Console
file_path = '/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10/tenpages/Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1824-1825.pdf'

def process_document_sample(
    project_id: str, location: str, processor_id: str, file_path: str, print_flag:bool
):
    from google.cloud import documentai_v1 as documentai
    from google.oauth2 import service_account

    client = documentai.DocumentProcessorServiceClient.from_service_account_json("/content/drive/MyDrive/Colab Notebooks/documentai/theorem-prover-4182022-62dfbc9d2f0f.json")

    # You must set the api_endpoint if you use a location other than 'us', e.g.:
    opts = {}
    if location == "us":
        opts = {"api_endpoint": "us-documentai.googleapis.com",
                "credentials_file":"/content/drive/MyDrive/Colab Notebooks/documentai/theorem-prover-4182022-62dfbc9d2f0f.json"}

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id
    # You must create new processors in the Cloud Console first
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

    with open(file_path, "rb") as image:
        image_content = image.read()

    # Read the file into memory
    document = {"content": image_content, "mime_type": "application/pdf"}

    # Configure the process request
    request = {"name": name, "raw_document": document}

    # Recognizes text entities in the PDF document
    result = client.process_document(request=request)

    document = result.document

    print("Document processing complete.")

    # For a full list of Document object attributes, please reference this page: https://googleapis.dev/python/documentai/latest/_modules/google/cloud/documentai_v1beta3/types/document.html#Document

    document_pages = document.pages

    # Read the text recognition output from the processor
    if print_flag:
      print("The document contains the following paragraphs:")
      for page in document_pages:
          paragraphs = page.paragraphs
          for paragraph in paragraphs:
              paragraph_text = get_text(paragraph.layout, document)
              print(f"Paragraph text: {paragraph_text}")
    return document


# Extract shards from the text field
def get_text(doc_element: dict, document: dict):
    """
    Document AI identifies form fields by their offsets
    in document text. This function converts offsets
    to text snippets.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in doc_element.text_anchor.text_segments:
        start_index = (
            int(segment.start_index)
            if segment in doc_element.text_anchor.text_segments
            else 0
        )
        end_index = int(segment.end_index)
        response += document.text[start_index:end_index]
    return response


Some of Helper methods from Google Document AI to exmaine the the Output & options.

In [ ]:

def print_page_dimensions(dimension: dict) -> None:
    print(f"    Width: {str(dimension.width)}")
    print(f"    Height: {str(dimension.height)}")


def print_detected_langauges(detected_languages: dict) -> None:
    print("    Detected languages:")
    for lang in detected_languages:
        code = lang.language_code
        conf_percent = "{:.1%}".format(lang.confidence)
        print(f"        {code} ({conf_percent} confidence)")


def print_paragraphs(paragraphs: dict, text: str) -> None:
    print(f"    {len(paragraphs)} paragraphs detected:")
    first_paragraph_text = layout_to_text(paragraphs[0].layout, text)
    print(f"        First paragraph text: {repr(first_paragraph_text)}")
    last_paragraph_text = layout_to_text(paragraphs[-1].layout, text)
    print(f"        Last paragraph text: {repr(last_paragraph_text)}")


def print_blocks(blocks: dict, text: str) -> None:
    print(f"    {len(blocks)} blocks detected:")
    first_block_text = layout_to_text(blocks[0].layout, text)
    print(f"        First text block: {repr(first_block_text)}")
    last_block_text = layout_to_text(blocks[-1].layout, text)
    print(f"        Last text block: {repr(last_block_text)}")


def print_lines(lines: dict, text: str) -> None:
    print(f"    {len(lines)} lines detected:")
    first_line_text = layout_to_text(lines[0].layout, text)
    print(f"        First line text: {repr(first_line_text)}")
    last_line_text = layout_to_text(lines[-1].layout, text)
    print(f"        Last line text: {repr(last_line_text)}")


def print_tokens(tokens: dict, text: str) -> None:
    print(f"    {len(tokens)} tokens detected:")
    first_token_text = layout_to_text(tokens[0].layout, text)
    first_token_break_type = tokens[0].detected_break.type_.name
    print(f"        First token text: {repr(first_token_text)}")
    print(f"        First token break type: {repr(first_token_break_type)}")
    last_token_text = layout_to_text(tokens[-1].layout, text)
    last_token_break_type = tokens[-1].detected_break.type_.name
    print(f"        Last token text: {repr(last_token_text)}")
    print(f"        Last token break type: {repr(last_token_break_type)}")


def layout_to_text(layout: dict, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirity of the document's text. This function converts
    offsets to a string.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in layout.text_anchor.text_segments:
        start_index = (
            int(segment.start_index)
            if segment in layout.text_anchor.text_segments
            else 0
        )
        end_index = int(segment.end_index)
        response += text[start_index:end_index]
    return response


### List of Papers:
We save JSON responses, blocks and paragraphs into seperate dictionaries.

In [ ]:
file_name = os.path.basename(tocs['Slave Trade Volume 10'][1])
class_a = tocs['Slave Trade Volume 10'][1]
file_paths = ['/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10/tenpages/class-a-tocs.pdf',
              '/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10/tenpages/class-b-tocs.pdf',
              '/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 11/tenpages/class-a-tocs.pdf',
              '/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 11/tenpages/class-b-tocs.pdf']

text_volumes = {'Slave Trade Volume 10':{"class_a":[], "class_b":[]}, 'Slave Trade Volume 11':{"class_a":[], "class_b":[]}}
text_blocks = {'Slave Trade Volume 10':{"class_a":[], "class_b":[]}, 'Slave Trade Volume 11':{"class_a":[], "class_b":[]}}

raw_responses = {'Slave Trade Volume 10':{"class_a":[], "class_b":[]}, 'Slave Trade Volume 11':{"class_a":[], "class_b":[]}}

for file_path in file_paths:
  if "Slave Trade Volume 10" in file_path:
    vol = "Slave Trade Volume 10"
  else:
    vol = "Slave Trade Volume 11"
  ten_pages = {}
  text_page_blocks = {}
  print("This Document Parsing is for %s" %(file_path))
  res = process_document_sample(project_id, location, processor_id, file_path, False)
  #print(res)
  with open(file_path, "rb") as image:
      image_content = image.read()

  # Read the file into memory
  doc = {"content": image_content, "mime_type": "application/pdf"}
  print("The document contains the following paragraphs:")
  document = res
  text = res.text
  #print(f"Full document text: {repr(text)}\n")
  print(f"There are {len(res.pages)} page(s) in this document.\n")
  this_coll = "class_a" if "class-a" in file_path else "class_b"
  raw_responses[vol][this_coll] = res

  for page in document.pages:
      print(f"Page {page.page_number}:")
      print_page_dimensions(page.dimension)

      counter = 0
      text_page_blocks[page.page_number] = []
      ten_pages[page.page_number] = []

      for block in page.blocks:

        counter += 1
        text_page_blocks[page.page_number].append({"block_number":counter, "text":layout_to_text(block.layout, text)})

      # print_detected_langauges(page.detected_languages)
      # print_paragraphs(page.paragraphs, text)
      # print_blocks(page.blocks, text)
      # print_lines(page.lines, text)
      # print_tokens(page.tokens, text)
      paragraphs = page.paragraphs
      para_counter = 0
      for paragraph in paragraphs:
          response = ""
          para_counter += 1
          # If a text segment spans several lines, it will
          # be stored in different text segments.
          for segment in paragraph.layout.text_anchor.text_segments:
              start_index = (
                  int(segment.start_index)
                  if segment in paragraph.layout.text_anchor.text_segments
                  else 0
              )
              end_index = int(segment.end_index)
              response += document.text[start_index:end_index]        #print(f"Paragraph text: {paragraph_text}")
          ten_pages[page.page_number].append({"paragraph_num":para_counter, "paragraph" : response})
  text_volumes[vol][this_coll].append(ten_pages)
  text_blocks[vol][this_coll].append(text_page_blocks)

This Document Parsing is for /content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10/tenpages/class-a-tocs.pdf
Document processing complete.
The document contains the following paragraphs:
There are 8 page(s) in this document.

Page 1:
    Width: 1758.0
    Height: 2275.0
Page 2:
    Width: 1758.0
    Height: 2275.0
Page 3:
    Width: 1758.0
    Height: 2275.0
Page 4:
    Width: 1758.0
    Height: 2275.0
Page 5:
    Width: 1758.0
    Height: 2275.0
Page 6:
    Width: 1758.0
    Height: 2275.0
Page 7:
    Width: 1758.0
    Height: 2275.0
Page 8:
    Width: 1758.0
    Height: 2275.0
This Document Parsing is for /content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10/tenpages/class-b-tocs.pdf
Document processing complete.
The document contains the following paragraphs:
There are 9 page(s) in this document.

Page 1:
    Width: 1758.0
    Height: 2275.0
Page 2:
    Width: 1758.0
    Height: 2275.0
Page 3:
    Width: 1758.0
    Height

check the AI results for List of Papers

In [ ]:
text_blocks['Slave Trade Volume 10']['class_a']

In [ ]:
text_volumes['Slave Trade Volume 10']['class_a']

Save JSON responses

In [ ]:
import json, codecs
import google.cloud

classes = ['class_a', 'class_b']
vols = ['Slave Trade Volume 10', 'Slave Trade Volume 11']
path = '/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10/tenpages'
for vol in vols:
  for cl in classes:
    with open(os.path.join(path, vol+"_"+cl+"_json.json"), 'w') as f:
      this_json_text = google.cloud.documentai_v1.Document.to_json(raw_responses[vol][cl])
      f.write(this_json_text)

In [ ]:
l = list(classa_docs.values())[0]
l[0], os.path.basename(classa_collections['Slave Trade Volume 10'][l[0]][0])[0:6].strip('_')

('Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1824-1825',
 'Part2')

In [ ]:
os.path.basename(classa_collections['Slave Trade Volume 10'][l[0]][0])[6:-4]

'Class A. Correspondence with the British commissioners at Sierra Leone, the Havannah, Rio de Janeiro, and Surinam, relating to the slave trade. 1824-1825'

In [ ]:
type(this_json_text)

str

Save Paragraphs responses to dictionary

In [ ]:

doc_text_volumes = {'Slave Trade Volume 10':{"class_a":{}, "class_b":{}}, 'Slave Trade Volume 11':{"class_a":{}, "class_b":{}}}
doc_text_blocks = {'Slave Trade Volume 10':{"class_a": {}, "class_b":{}}, 'Slave Trade Volume 11':{"class_a":{}, "class_b":{}}}

doc_raw_responses = {'Slave Trade Volume 10':{"class_a":{}, "class_b":{}}, 'Slave Trade Volume 11':{"class_a":{}, "class_b":{}}}

this_coll = "class_a"

for vol, docs in classa_collections.items():

  for doc_name, file_paths in docs.items():
      doc_raw_responses[vol][this_coll][doc_name] = []
      doc_text_volumes[vol][this_coll][doc_name] = []
      doc_text_blocks[vol][this_coll][doc_name] = []

      for file_path in file_paths:
          partition = os.path.basename(file_path)[0:6].strip('_')
          print("Part %s for Collection name is %s"%(partition, doc_name ))

          ten_pages = {}
          text_page_blocks = {}
          print("This Document Parsing is for %s" %(file_path))
          res = process_document_sample(project_id, location, processor_id, file_path, False)
          #print(res)
          with open(file_path, "rb") as image:
              image_content = image.read()

          # Read the file into memory
          doc = {"content": image_content, "mime_type": "application/pdf"}
          print("The document contains the following paragraphs:")
          document = res
          text = res.text
          #print(f"Full document text: {repr(text)}\n")
          print(f"There are {len(res.pages)} page(s) in this document.\n")

          doc_raw_responses[vol][this_coll][doc_name].append({partition:google.cloud.documentai_v1.Document.to_json(res)})

          for page in document.pages:
              print(f"Page {page.page_number}:")
              print_page_dimensions(page.dimension)

              counter = 0
              text_page_blocks[page.page_number] = []
              ten_pages[page.page_number] = []

              for block in page.blocks:

                counter += 1
                text_page_blocks[page.page_number].append({"block_number":counter, "text":layout_to_text(block.layout, text)})
              paragraphs = page.paragraphs
              para_counter = 0
              for paragraph in paragraphs:
                  response = ""
                  para_counter += 1
                  # If a text segment spans several lines, it will
                  # be stored in different text segments.
                  for segment in paragraph.layout.text_anchor.text_segments:
                      start_index = (
                          int(segment.start_index)
                          if segment in paragraph.layout.text_anchor.text_segments
                          else 0
                      )
                      end_index = int(segment.end_index)
                      response += document.text[start_index:end_index]
                  ten_pages[page.page_number].append({"paragraph_num":para_counter, "paragraph" : response})
          doc_text_volumes[vol][this_coll][doc_name].append({partition:ten_pages})
          doc_text_blocks[vol][this_coll][doc_name].append({partition:text_page_blocks})


In [ ]:
import csv

def save_csv(this_dict : dict, file_name:str, volume:str, path:str, csv_columns:list):

  assert file_name, "%s Filename is empty!" %(file_name)
  print(path, volume)
  directory = os.path.join(path, volume)
  # if not os.path.isdir(directory):
  #   os.mkdir(directory)
  print(volume, file_name)
  file_path = os.path.join(path, file_name)
  try:
      with open(file_path, 'w') as csvfile:
          writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
          writer.writeheader()
          for data in this_dict:
              writer.writerow(data)
  except IOError:
      print("I/O error")
      return False
  return True

Save List of Papers Blocks & Paragraphs into CSV files

In [ ]:
import pandas as pd
for vol, this_coll_dict in text_volumes.items():
  for coll, pages in this_coll_dict.items():
    this_file = ""
    print("pages",len(pages[0]))
    if vol in list(tocs.keys()):
      toc_coll = tocs[vol]
    for c in toc_coll:
      if coll.replace('_', '-') in c:
        this_file = c
    print(this_file)
    print(len(list(pages[0].values())))
    this_pages = {}
    for page, p_list in pages[0].items():
      print(p_list[:2])
      save_csv(p_list, os.path.basename(this_file)[:-4]+"_"+str(page)+".csv", "apiresults-19", os.path.dirname(this_file), ['paragraph_num', 'paragraph'])

In [ ]:
import pandas as pd
for vol, this_coll_dict in text_blocks.items():
  for coll, pages in this_coll_dict.items():
    this_file = ""
    print("pages",len(pages[0]))
    if vol in list(tocs.keys()):
      toc_coll = tocs[vol]
    for c in toc_coll:
      if coll.replace('_', '-') in c:
        this_file = c
    #print(vol, coll)
    print(this_file)
    print(len(list(pages[0].values())))
    this_pages = {}
    for page, p_list in pages[0].items():
      print(len(p_list))
      this_pages[page] = []
      this_pages[page].append(p_list)
      print(p_list[:2])
      save_csv(p_list, os.path.basename(this_file)[:-4]+"_"+str(page)+"_blocks.csv", "apiresults-19", os.path.dirname(this_file),
               ['block_number', 'text'])
    # for page_num, paragraphs in pages[0].items():
    #   for paragraph in paragraphs:
    #     print(paragraph)

In [ ]:
tocs

{'Slave Trade Volume 10': ['/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10/tenpages/class-a-tocs.pdf',
  '/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10/tenpages/class-b-tocs.pdf'],
 'Slave Trade Volume 11': ['/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 11/tenpages/class-a-tocs.pdf',
  '/content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 11/tenpages/class-b-tocs.pdf']}

Save Document text JSON into JSON files

In [ ]:
path = ''
vols = ['Slave Trade Volume 10', 'Slave Trade Volume 11']

for vol in doc_raw_responses.keys():
  for cl in doc_raw_responses[vol].keys():
    for doc, partitions_json_texts in doc_raw_responses[vol][cl].items():
      path = volumes_dict[vol]
      print(vol, path, doc)
      directory = os.path.join(path, 'tenpages', 'apiresults-19', 'documents', doc)
      if not os.path.isdir(directory):
        os.mkdir(directory)
      print(len(partitions_json_texts))
      for part_json_text_dict in partitions_json_texts:
        print(next(iter(part_json_text_dict.keys())))
        part_num, this_json_text = next(iter(part_json_text_dict.items()))
        with open(os.path.join(directory, part_num+"_"+doc+".json"), 'w') as f:
          f.write(this_json_text)
          print(part_num)

In [ ]:
for vol in doc_raw_responses.keys():
  for cl in doc_raw_responses[vol].keys():
    for doc, partitions_json_texts in doc_raw_responses[vol][cl].items():
      path = volumes_dict[vol]
      print(vol, path)
      directory = os.path.join(path, 'tenpages', 'apiresults-19', 'documents', doc, 'response_json')
      print(len(partitions_json_texts) == len(os.listdir(directory)))

Slave Trade Volume 10 /content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10
True
Slave Trade Volume 10 /content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 10
True
Slave Trade Volume 11 /content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 11
True
Slave Trade Volume 11 /content/drive/MyDrive/Colab Notebooks/slave-trade-volumes/all/Slave Trade Volume 11
True


Save paragraphs into CSV files

In [ ]:
csv_columns = ['page', 'paragraph', 'para_num']

for vol, this_coll_dict in doc_text_volumes.items():
  for coll in this_coll_dict.keys():
      for doc, partitions in doc_text_volumes[vol][coll].items():
        this_file = ""
        true_pg_num = 1
        all_partitions = []
        print("partitions",len(partitions))
        count_p = len(partitions)

        path = os.path.join(volumes_dict[vol], 'tenpages', 'apiresults-19', 'documents', doc)
        this_file = os.path.join(path, 'paragraphs')
        #print(this_file)

        for partition in partitions:
          this_partition = []
          part_num, pages = next(iter(partition.items()))
          #print(pages.keys(), pages[1])
          for page_num, page in pages.items():
            print(page_num)

            for para in page:
              this_partition.append({"page": page_num, "para_num":para['paragraph_num'], "paragraph":para['paragraph']})
              all_partitions.append({"page": true_pg_num, "para_num":para['paragraph_num'], "paragraph":para['paragraph']})
            true_pg_num += 1
          #print(os.path.dirname(this_file))
            #print(page_num)
          print("length of dict",len(this_partition))
          save_csv(this_partition, str(part_num)+"_"+doc+".csv", "apiresults-19", this_file, csv_columns)
        print(page_num, true_pg_num)
        save_csv(all_partitions, doc+".csv", "apiresults-19", os.path.dirname(this_file), csv_columns)

          ##print(json.dumps(partition))
          #pages = list(partitions[0].values())[0].keys()
        #print(list(partitions[0].values())[0][1][0].keys())

Save Blocks into CSV files

In [ ]:
csv_columns = ['page', 'block_num', 'block']
for vol, this_coll_dict in doc_text_blocks.items():
  for coll in this_coll_dict.keys():
      for doc, partitions in doc_text_blocks[vol][coll].items():
        this_file = ""
        print("partitions",len(partitions))
        true_pg_num = 1
        all_partitions = []

        path = os.path.join(volumes_dict[vol], 'tenpages', 'apiresults-19', 'documents', doc)
        this_file = os.path.join(path, 'blocks')
        print(this_file)

        for partition in partitions:
          this_partition = []
          part_num, pages = next(iter(partition.items()))
          for page_num, blocks in pages.items():
            print(len(blocks))

            for block in blocks:
              print(block['block_number'])
              this_partition.append({"page": page_num, "block_num":block['block_number'], "block":block['text']})
              all_partitions.append({"page": true_pg_num, "block_num":block['block_number'], "block":block['text']})
            true_pg_num += 1
          save_csv(this_partition, str(part_num)+"_"+doc+".csv", "apiresults-19", this_file, csv_columns)
        print(page_num, true_pg_num)
        save_csv(all_partitions, "blocks_"+doc+".csv", "apiresults-19", os.path.dirname(this_file), csv_columns)

In [ ]:
l = list(classa_docs.values())[0]
doc_text_blocks['Slave Trade Volume 10']['class_a'][l[0]][0]

{'Part2': {1: [{'block_number': 1,
    'text': 'SIERRA LEONE.\n(General.)\ntrade is legally pursued, but it may be thought that where the trade is pro-\nscribed it would not be dared to be carried on by Portuguese subjects.\nIn the course of the last year some boats from Bissao, it is said, landed\nat the mouth of the Rio Nunez; sacked some of the villages there, and\ncarried off to Bissao to be sold into Slavery as many of their inhabitants as they\ncould take.\n'},
   {'block_number': 2, 'text': '4\n'},
   {'block_number': 3,
    'text': 'Besides the barbarity of this practice, its consequence is, that the natives\nwithin the reach of such kidnapping expeditions, are rendered savage and\nuntractable, so much so that they are always disposed to deal harshly with such\nEuropeans as may fall into their hands. The natives of one of the Bisago\nIslands, which lie about the mouth of the River on which Bissao is situated,\npossessed themselves of a boat with its crew belonging to the Spanis